In [1]:
import os
import gym
import gym_donkeycar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import time
import pickle
#import birds_eye_vector_space
import basis_my_cv
import random
from pandas import Series, DataFrame
from collections import deque

#from keras.layers import Dense
#from tensorflow.keras.optimizers import Adam
#from keras.initializers import normal, identity
#from keras.models import model_from_json
#from keras.models import Sequential
#from keras.layers.core import Dense, Dropout, Activation, Flatten
#from keras.layers.convolutional import Convolution2D, MaxPooling2D

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.client import device_lib
#from keras import backend as K
#import tensorflow.keras.backend as K

C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find d

In [2]:
# use gpu
#K.tensorflow_backend._get_available_gpus()
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
#K.tensorflow_backend._get_available_gpus()

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [4]:
tf.test.is_built_with_cuda()

True

In [5]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [7]:
# run on the second GPU
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
#os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [8]:
len(tf.config.list_physical_devices('GPU'))
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7224688915679964692
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9401860096
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 267470071666152665
 physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:49:00.0, compute capability: 7.5"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6271991808
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14637973241787989751
 physical_device_desc: "device: 1, name: GeForce RTX 2070 SUPER, pci bus id: 0000:21:00.0, compute capability: 7.5"
 xla_global_id: 2144165316]

In [9]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

In [10]:
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [11]:
# Supress scientific notation like: e*+03
np.set_printoptions(suppress=True)

In [12]:
ANCHOR_POINTS = 8
ROAD_ROI = np.array([(120,90),(200,90),(0,200),(320,200)],dtype='float32')
WARPED_IMAGE_SHAPE = np.array([[10,320],[0,0],[200,0],[200,310]],np.int32)            # NEW IMAGE Shape after Warping !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
WARPED_IMAGE_HEIGHT = 320
WARPED_IMAGE_WIDTH = 200
NUMBER_OF_DEPTH_LAYERS = 50

VECTOR_SPACE_IMAGE_ROWS = 80
VECTOR_SPACE_IMAGE_COLUMNS = 80
VECTOR_SPACE_IMAGE_CHANNELS = 4 # 4*(50,100) stacked frames

In [13]:
# function to save the plots
def save_plot(data, name, episode):
    plt.figure(figsize=(8,5), frameon=True)
    #plt.plot([episode for episode in range(len(data))], data)
    plt.plot([ep for ep in range(episode)], data)
    plt.xlabel('Episodes')
    plt.ylabel(name)
    figplot = '%s-episode-%s.png' % (name,episode)
    #figplot = '{}-episode-{}.png'.format(name,episode)
    #location = '{}/saves/torch/{}'.format(os.getcwd(),figplot)
    #location = '%s/collected_data/plots/%s' % (os.getcwd(),figplot)
    location = 'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\collected_data\\plots\\%s' % figplot 
    plt.savefig(location, transparent=False)
    plt.close()

In [14]:
def save_data_as_dataframe(episode_number, reward, loss, measured_time, max_cte, average_cte, last_lap_time):
    episodes = [i for i in range(1, episode_number+1)]
    collected_data = {'episode': episodes, 'reward': reward, 'loss':loss, 'time': measured_time, 'max_cte': max_cte, 'average_cte': average_cte, 'lap_time': last_lap_time}
    df_data = DataFrame.from_dict(collected_data).set_index('episode')
    
    df_name = 'data-episode-%s.pkl' % episode
    #location = '%s/collected_data/raw_data/%s' % (os.getcwd(),df_name)
    location = 'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\collected_data\\raw_data\\%s' % df_name
    df_data.to_pickle(location)

In [15]:
#vector_space = birds_eye_vector_space.Vector_space(ANCHOR_POINTS, ROAD_ROI, WARPED_IMAGE_SHAPE, WARPED_IMAGE_HEIGHT, WARPED_IMAGE_WIDTH, NUMBER_OF_DEPTH_LAYERS)

In [16]:
os.getcwd()

'C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\Master-Thesis-Development-of-a-Deep-RL-Model-for-simulated-Driving-2D-Vector-Space\\docker\\src'

In [17]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
        self.t = 0
        self.max_Q = 0
        self.train = True
        # Set to True to train on images with segmented lane lines
        self.lane_detection = False
        
        # Huber loss
        self.huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
        
        # Get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        
        # CNN Input
        #self.input_shape = (VECTOR_SPACE_IMAGE_CHANNELS, VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, 4)   # == 4 * (50, 100, 1); keras: (1,50,100,4)
        self.input_shape = (1, VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, VECTOR_SPACE_IMAGE_CHANNELS)   # == 4 * (50, 100, 1); keras: (1,50,100,4)
        
        # These are hyper parameters for the DQN
        self.discount_factor = 0.99
        self.learning_rate = 1e-4
        if (self.train):
            self.epsilon = 1.0
            self.initial_epsilon = 1.0
        else:
            self.epsilon = 1e-6
            self.initial_epsilon = 1e-6
        self.epsilon_min = 0.02
        self.batch_size = 512
        self.train_start = 100
        self.explore = 10000
        
        # Create replay memory using deque
        self.memory = deque(maxlen=10000)
        
        # Create main model and target model                # Double DQN !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        self.model = self.build_model()
        self.target_model = self.build_model()

        # Copy the model to target model
        # --> initialize the target model so that the parameters of model & target model to be same
        self.update_target_model()
        
    def build_model_old(self):
        print('delte this')
        #model = Sequential()
        #model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(VECTOR_SPACE_IMAGE_ROWS,VECTOR_SPACE_IMAGE_COLUMNS,VECTOR_SPACE_IMAGE_CHANNELS)))  #80*80*4
        #model.add(Activation('relu'))
        #model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
        #model.add(Activation('relu'))
        #model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
        #model.add(Activation('relu'))
        #model.add(Flatten())
        #model.add(Dense(512))
        #model.add(Activation('relu'))

        ## 15 categorical bins for Steering angles
        #model.add(Dense(15, activation="linear")) 

        #adam = Adam(lr=self.learning_rate)
        #model.compile(loss='mse',optimizer=adam)

        #return model

    def build_model(self):
        model = keras.Sequential([
            layers.Conv2D(filters=32, strides=(4, 4),kernel_size=8, padding='same', activation='relu', input_shape=(self.input_shape[1:])),
            layers.Conv2D(filters=64, strides=(2, 2),kernel_size=4, padding='same', activation='relu'),
            layers.Conv2D(filters=64, strides=(1, 1),kernel_size=3, padding='same', activation='relu'),
            layers.Flatten(),
            layers.Dense(units=512,activation='relu'),
            layers.Dense(units=512,activation='relu'), # TODO ADDED THIS LAYER FOR TESTING
            layers.Dense(units=15,activation='linear'),
        ])
        #optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        optimizer = tf.keras.optimizers.Adam()
        #optimizer = tf.keras.optimizers.SGD()
        model.compile(loss='mse', optimizer=optimizer)
        #model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=optimizer)
        return model
    
    def calculate_loss(self, y_true, y_pred):    
        y_pred = tf.convert_to_tensor_v2(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)
        return tf.reduce_mean(math_ops.square(y_pred - y_true), axis=-1)
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # Get action from model using epsilon-greedy policy
    def get_action(self, s_t):
        if np.random.rand() <= self.epsilon:
            #print("Return Random Value")
            #return random.randrange(self.action_size)
            return np.random.uniform(-1,1)
        else:
            #print("Return Max Q Prediction")
            q_value = self.model.predict(s_t)
            # Convert q array to steering value
            return linear_unbin(q_value[0])

    def replay_memory(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        #if self.epsilon > self.epsilon_min:
        #    #self.epsilon *= self.epsilon_decay
        #    self.epsilon -= (self.initial_epsilon - self.epsilon_min) / self.explore

    def train_replay(self):
        if len(self.memory) < self.train_start:
            return
        
        batch_size = min(self.batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)

        state_t, action_t, reward_t, state_t1, terminal = zip(*minibatch)
        #print('Starting training\nstate_t: %s\naction_t: %s\nreward_t: %s\nstate_1: %s\nterminal: %s' % (len(state_t), len(action_t), len(reward_t), len(state_t1), len(terminal)))
        state_t = np.concatenate(state_t)
        state_t1 = np.concatenate(state_t1)
        #print('Concat:\nstate: %s\nstate_t1: %s' % (state_t, state_t1))
        targets = self.model.predict(state_t)
        self.max_Q = np.max(targets[0])
        target_val = self.model.predict(state_t1)
        target_val_ = self.target_model.predict(state_t1)
        for i in range(batch_size):
            if terminal[i]:
                targets[i][action_t[i]] = reward_t[i]
            else:
                a = np.argmax(target_val[i])
                targets[i][action_t[i]] = reward_t[i] + self.discount_factor * (target_val_[i][a])

        
        # train on the current batch and return a dictionary with the loss and so on
        #metrics = self.model.train_on_batch(x=state_t, y=targets, reset_metrics=True, return_dict=True)           
        metrics = self.model.train_on_batch(x=state_t, y=targets, reset_metrics=True, return_dict=True) #reset Metrix different
        return metrics['loss']
        
    def load_model(self, name):
        self.model.load_weights(name)

    # Save the model which is under training
    def save_model(self, name):
        self.model.save_weights(name)

In [18]:
def linear_bin(a):
    """
    Convert a value to a categorical array.

    Parameters
    ----------
    a : int or float
        A value between -1 and 1

    Returns
    -------
    list of int
        A list of length 15 with one item set to 1, which represents the linear value, and all other items set to 0.
    """
    a = a + 1
    b = round(a / (2 / 14))
    arr = np.zeros(15)
    arr[int(b)] = 1
    return arr

In [19]:
def linear_unbin(arr):
    """
    Convert a categorical array to value.

    See Also
    --------
    linear_bin
    """
    if not len(arr) == 15:
        raise ValueError('Illegal array length, must be 15')
    b = np.argmax(arr)
    a = b * (2 / 14) - 1
    return a

In [20]:
def process_image(obs):
    #obs = cv.cvtColor(obs, cv.COLOR_BGR2GRAY)
    obs = cv.resize(obs, (VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS))
    edges = basis_my_cv.detect_edges(obs, low_threshold=50, high_threshold=150)
    rho = 0.8
    theta = np.pi/180
    threshold = 25
    min_line_len = 5
    max_line_gap = 10
    hough_lines = basis_my_cv.hough_lines(edges, rho, theta, threshold, min_line_len, max_line_gap)
    left_lines, right_lines = basis_my_cv.separate_lines(hough_lines)
    filtered_right, filtered_left = [],[]
    if len(left_lines):
        filtered_left = basis_my_cv.reject_outliers(left_lines, cutoff=(-30.0, -0.1), lane='left')
    if len(right_lines):
        filtered_right = basis_my_cv.reject_outliers(right_lines,  cutoff=(0.1, 30.0), lane='right')
    lines = []
    if len(filtered_left) and len(filtered_right):
        lines = np.expand_dims(np.vstack((np.array(filtered_left),np.array(filtered_right))),axis=0).tolist()
    elif len(filtered_left):
        lines = np.expand_dims(np.expand_dims(np.array(filtered_left),axis=0),axis=0).tolist()
    elif len(filtered_right):
        lines = np.expand_dims(np.expand_dims(np.array(filtered_right),axis=0),axis=0).tolist()
    ret_img = np.zeros((80,80))
    if len(lines):
        try:
            basis_my_cv.draw_lines(ret_img, lines, thickness=1)
        except:
            pass
    return ret_img

## Main training loop

In [ ]:
#%% SET UP ENVIRONMENT
# Normal one 
#os.environ['DONKEY_SIM_PATH'] = "/home/zamy/masterthesis/DonkeySimLinux/donkey_sim.x86_64"
os.environ['DONKEY_SIM_PATH'] = "C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\test\\DonkeySimWin\\donkey_sim.exe"
os.environ['DONKEY_SIM_PORT'] = str(9091)
os.environ['DONKEY_SIM_HEADLESS'] = str(0) # "1" is headless

CAMERA_CONF = {'cam_config':{'img_w': '320',
                        'img_h': '200',
                        'img_d': '1',   # 3 for colored Tensor image
                        'img_enc': 'PNG', 
                        'fov': '90', 
                        'fish_eye_x': '0.0', 
                        'fish_eye_y': '0.0', 
                        'offset_x': '0.0', 
                        'offset_y': '0.0', 
                        'offset_z': '0.0', 
                        'rot_x': '0'}}
# Other tracks
#env = gym.make("donkey-generated-track-v0",conf=CAMERA_CONF)

# TRACKS TRACKS - TKarmer Tracks
#env = gym.make("donkey-generated-track-v0",conf=CAMERA_CONF) #,conf=config
# Random track, but the reward is kinda not working, IF THE REWARD WORKS IT IS KINDA RANDOM !!, this maybe debends on the environment
#env = gym.make("donkey-generated-roads-v0",conf=CAMERA_CONF) #,conf=config
env = gym.make("donkey-generated-roads-v0") #,conf=config

# Mini Monaco Track
#env = gym.make("donkey-minimonaco-track-v0",conf=CAMERA_CONF) #,conf=config
#env = gym.make("donkey-generated-track-v0") #,conf=config
#env = gym.wrappers.ResizeObservation(env,(200,320))

## DELETE LATER
#env.frameskip = 1
#gym.wrappers.max

# Create DQN Model
# Get size of state and action from environment
state_size = (VECTOR_SPACE_IMAGE_ROWS, VECTOR_SPACE_IMAGE_COLUMNS, VECTOR_SPACE_IMAGE_CHANNELS)
action_size = 15 # Steering and Throttle
throttle = 0.15 # Set the throttle as a constant value
agent = DQNAgent(state_size, action_size)


# Arrays for data collection / exploratory data analysis
average_cte = []
average_speed = []
measured_time = []
average_rewards = []
average_loss = []
collected_max_cte = []
collected_lap_time = []

# when a model and the data should be saved
save_state = 200

for episode in range(1,7001):
    # Resetting the environment and preprocessing the first image
    observation = env.reset()
    # Graykonvertion of the observation image
    #observation = cv.cvtColor(observation, cv.COLOR_BGR2GRAY)
    #observation = vector_space.image_preprocessing(observation)
    observation = process_image(observation)
    # frame stacking (4 times)
    obv_stack = np.stack((observation, observation, observation, observation), axis= 2)
    # reshaping for keras
    obv_stack = obv_stack.reshape(1, obv_stack.shape[0], obv_stack.shape[1], obv_stack.shape[2])
    
    #action = np.array([0,0.10]) # drive straight with small speed
    #action = np.array([1,0.1]) # drive straight with small speed
    
    # summed up values for data collection
    # cumulative values for data collection
    total_cte = float(0)
    total_speed = float(0)
    total_reward = float(0)
    total_time = time.time()
    total_loss = float(0)
    max_cte = int(0)
    lap_time = int(0)
        
    # counting the amount of frames per episode
    frames = int(0)
    
    # boolean that describes if the env is done with this episode
    done = False
    while not done:
        # incrementing the amount of frames per episode
        frames += 1
        
        # Making a prediction for the current state
        # and getting the information form the next step
        steering = agent.get_action(obv_stack)
        action = [steering, throttle]
        next_observation, reward, done, info = env.step(action)
        
        current_cte = abs(info['cte'])
        # punish and reward for a low or high 'CTE' with the logarithmic function
        if current_cte < .25:
            reward += abs(np.log(.10)) * 8  # 4 before
        elif current_cte >= .25 and current_cte < .5:
            reward += current_cte * 2
        elif current_cte >= .5 and current_cte < 1: 
            reward += abs(np.log(current_cte))
        elif current_cte >= 1 and current_cte < 2: 
            reward -= abs(current_cte) * 2
        elif current_cte >= 2 and current_cte < 3: 
            reward -= abs(current_cte) * 3
        elif current_cte >= 3 and current_cte < 4: 
            reward -= abs(current_cte) * 4
        elif current_cte >=4:
            reward -= abs(current_cte) * 5
                    
        # if the car leaves the track punish it
        if done:
            reward -= 100
        # Check if the CTE of the environment is bugged
        if episode == 1:
            #print('CTE: %s\tReward: %s' % (info['cte'], reward)
            print('REWARD: %s\tCTE: %s' % (reward, current_cte))
        
        # Graykonvertion of the observation image
        next_observation = cv.cvtColor(next_observation, cv.COLOR_BGR2GRAY)
        
        # preprocessing the new observation
        #next_observation_show = vector_space.image_preprocessing(next_observation)
        next_observation_show = process_image(next_observation)
        # reshaping for keras
        #next_observation = next_observation_show.reshape(1, next_observation_show.shape[0], next_observation_show.shape[1], 1)
        # appending to the observation stack
        #obv_stack = np.append(next_observation, obv_stack[:, :, :, :3], axis=3)
        
        obv_stack_t1 = np.stack([next_observation_show, obv_stack[0,:,:,0],obv_stack[0,:,:,1],obv_stack[0,:,:,2]], axis=2)
        obv_stack_t1 = obv_stack_t1.reshape(1, obv_stack_t1.shape[0], obv_stack_t1.shape[1], obv_stack_t1.shape[2])
                
        # saving the sample <s, a, r, s'> to the replay memory
        agent.replay_memory(obv_stack, np.argmax(linear_bin(steering)), reward, obv_stack_t1, done)        
        
        # training the DDQN, if training is enabled
        #if agent.train:
        #    loss = agent.train_replay()
        #    if loss != None:
        #        total_loss += loss
        #        #print('loss: %s ' % (total_loss / frames))
        
        # overwriting the stack and incrementing the time/frame counter
        obv_stack = obv_stack_t1
        agent.t += 1       
        
        # adding up the collected data
        current_cte = info['cte'] 
        total_cte += current_cte
        total_speed += info['speed']
        total_reward += reward
        # update if there is any change in incrementation 
        if max_cte < current_cte:
            max_cte = current_cte
        if lap_time < info['last_lap_time']:
            lap_time = info['last_lap_time']
            
        cv.imshow('vec img', next_observation_show)
        
        # stop the training if the car passes 120 seconds,
        # so there is no infinit learning
        if abs(total_time - time.time()) >= 120:
            done = True

        # Graykonvertion of the observation image
        #observation = cv.cvtColor(observation, cv.COLOR_BGR2GRAY)
        
        # Image preprocessing
        #vec_image = vector_space.image_preprocessing(observation)
        
        #print('REWARD: %s' % reward)
        
        #cv.imshow('vec img', vec_image)
        
        #break       # TODO: DELTE LATER
        if cv.waitKey(25) & 0xFF == ord('q'):
            cv.destroyAllWindows()
            break
    
    # Training after each episode
    # do 10 training sets of each with a batch size of 512
    for i in range(0,10):
        if agent.train:
            total_loss += agent.train_replay()
            #total_loss = agent.train_replay()
            #if loss != None:
            #    total_loss = loss
    
    
    # AFTER this Episode & the environment returns True for the done variable
    # updating the target DDQN
    agent.update_target_model()
    
    # calculating the average loss in this episode
    #episode_loss = total_loss.copy() #/ frames
    total_loss = total_loss / 512 * 10
    episode_cte = total_cte / frames
    episode_reward = total_reward / frames
    
    # updating the time parameter
    total_time = abs(total_time - time.time())
    
    # appending the collected data
    average_cte.append(episode_cte)
    average_speed.append(total_speed / frames)
    measured_time.append(total_time)
    average_rewards.append(episode_reward)
    average_loss.append(total_loss)
    collected_max_cte.append(max_cte)
    collected_lap_time.append(lap_time) 
    
    # Testing of saving the plot
    #if episode % 3 == 0 and episode != 0:
    #    save_plot(colleted_rewards, 'Reward', episode)
    #    cv.destroyAllWindows()
    #    break
    
    # Print episode information    
    print('EPISODE: %s | TIME: %s s | REWARD: %s | FRAMES: %s | QMAX: %s | EPSILON: %s | CTE: %s | LOSS: %s' % (episode, total_time, episode_reward, frames, str(agent.max_Q), agent.epsilon, episode_cte, total_loss))    
    
    # update the epsilon after each episode
    # Epsilion decay over time/amount of trainings
    if agent.epsilon >= agent.epsilon_min:
        #self.epsilon *= self.epsilon_decay
        #self.epsilon -= (self.initial_epsilon - self.epsilon_min) / self.explore
        #self.epsilon -= 0.00025
        agent.epsilon -= 0.00025 # before 0.00035
    
    if episode % save_state == 0 and episode != 0:
        print('saving after %s episodes' % save_state)
        # saving the model
        agent.save_model('C:\\Users\\studwilksa2535\\Desktop\\DonkeyCarAI\\models\\model_episode_%s.h5' % episode)        
        
        # Saving the data as plots
        save_plot(average_rewards, 'Reward', episode)
        save_plot(average_cte, 'Average cte', episode)
        save_plot(collected_max_cte, 'Max cte', episode)
        save_plot(measured_time, 'Time', episode)
        save_plot(average_loss, 'Loss', episode)
        save_plot(collected_lap_time, 'Lap Time', episode)
        
        # Saving the collected data
        save_data_as_dataframe(episode, average_rewards, average_loss, measured_time, collected_max_cte, average_cte, collected_lap_time)
    
    # closing all cv windows
    cv.destroyAllWindows()

# closing all cv windows
cv.destroyAllWindows()
# Close the enviroment
env.close()

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
C:\Users\studwilksa2535\AppData\Roaming\Python\Python38\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 5.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: port 9091
loading scene generated_road


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.


REWARD: 18.42068227068214	CTE: 4.658727e-05
REWARD: 18.422150181014587	CTE: 5.42303e-05
REWARD: 18.50215732861931	CTE: 0.0001973086
REWARD: 18.583280266344673	CTE: 2.698664e-05
REWARD: 18.65591459161854	CTE: 0.0004304353
REWARD: 18.73335047112035	CTE: 0.0006311428
REWARD: 18.80249120043712	CTE: 0.001318939
REWARD: 18.874214488772594	CTE: 0.001780437
REWARD: 18.936897459650638	CTE: 0.003461112
REWARD: 19.006660128506756	CTE: 0.005121341
REWARD: 19.077751088158635	CTE: 0.005758009
REWARD: 19.144510256729017	CTE: 0.004943698
REWARD: 19.21004678338141	CTE: 0.001873337
REWARD: 19.279701590101865	CTE: 0.001896521
REWARD: 19.34788871826911	CTE: 0.005160315
REWARD: 19.399360676321603	CTE: 0.004767529
REWARD: 19.467241302642314	CTE: 0.002480428
REWARD: 19.534104621741236	CTE: 0.0006067133
REWARD: 19.59578113131672	CTE: 0.002299225
REWARD: 19.660347196691855	CTE: 0.006519644
REWARD: 19.700956660307135	CTE: 0.006210014
REWARD: 19.745615665330398	CTE: 0.006271973
REWARD: 19.825156862976044	CTE: 0.

C:\Users\studwilksa2535\Desktop\DonkeyCarAI\Master-Thesis-Development-of-a-Deep-RL-Model-for-simulated-Driving-2D-Vector-Space\docker\src\basis_my_cv.py:30: RuntimeWarning: divide by zero encountered in int_scalars
  return (y1 - y2) / (x1 - x2)


...

EPISODE: 5632 | TIME: 129.34845542907715 s | REWARD: -6.688548170406421 | FRAMES: 933 | QMAX: 1419.7637 | EPSILON: 0.019750000000082396 | CTE: -2.7776953630849954 | LOSS: 32.64635294675827
EPISODE: 5633 | TIME: 60.99790143966675 s | REWARD: 18.41720368416134 | FRAMES: 392 | QMAX: 1317.0718 | EPSILON: 0.019750000000082396 | CTE: 0.12436958502015308 | LOSS: 32.88464084267616
EPISODE: 5634 | TIME: 129.26923036575317 s | REWARD: 5.2704298945250905 | FRAMES: 932 | QMAX: 1387.1416 | EPSILON: 0.019750000000082396 | CTE: -1.396423502292808 | LOSS: 21.098967790603638
EPISODE: 5635 | TIME: 129.37736582756042 s | REWARD: 5.3590667643760606 | FRAMES: 926 | QMAX: 1362.5726 | EPSILON: 0.019750000000082396 | CTE: -1.3061492860845583 | LOSS: 20.71796953678131
EPISODE: 5636 | TIME: 129.26843214035034 s | REWARD: -7.015780551135635 | FRAMES: 934 | QMAX: 1362.7147 | EPSILON: 0.019750000000082396 | CTE: -2.758016915107175 | LOSS: 18.584594875574112
EPISODE: 5637 | TIME: 129.1910355091095 s | REWARD: 17.05

EPISODE: 5676 | TIME: 116.60491895675659 s | REWARD: -9.70630919209774 | FRAMES: 830 | QMAX: 1237.5731 | EPSILON: 0.019750000000082396 | CTE: -3.2764508800261445 | LOSS: 34.36344876885414
EPISODE: 5677 | TIME: 17.48429036140442 s | REWARD: -0.5736548342869641 | FRAMES: 62 | QMAX: 1529.9353 | EPSILON: 0.019750000000082396 | CTE: -1.4634256087580642 | LOSS: 34.54663619399071
EPISODE: 5678 | TIME: 106.74101281166077 s | REWARD: 7.885651733048095 | FRAMES: 764 | QMAX: 1434.0597 | EPSILON: 0.019750000000082396 | CTE: -0.5391952473981677 | LOSS: 33.206610679626465
EPISODE: 5679 | TIME: 129.22719526290894 s | REWARD: 6.1540776863987094 | FRAMES: 926 | QMAX: 1614.6669 | EPSILON: 0.019750000000082396 | CTE: 0.8709230004111217 | LOSS: 29.839115887880325
EPISODE: 5680 | TIME: 20.954281330108643 s | REWARD: 0.060803498738394045 | FRAMES: 89 | QMAX: 1515.4167 | EPSILON: 0.019750000000082396 | CTE: -1.439955870338202 | LOSS: 46.17292910814285
EPISODE: 5681 | TIME: 129.26853466033936 s | REWARD: 10.6

EPISODE: 5720 | TIME: 93.96606731414795 s | REWARD: -3.897038680476889 | FRAMES: 651 | QMAX: 1379.4913 | EPSILON: 0.019750000000082396 | CTE: -2.356751981184947 | LOSS: 25.667129904031754
EPISODE: 5721 | TIME: 129.33744168281555 s | REWARD: 17.567386196514434 | FRAMES: 926 | QMAX: 1377.7747 | EPSILON: 0.019750000000082396 | CTE: -0.049323804266954735 | LOSS: 22.576739937067032
EPISODE: 5722 | TIME: 129.22961473464966 s | REWARD: 8.620060134581626 | FRAMES: 926 | QMAX: 1408.1783 | EPSILON: 0.019750000000082396 | CTE: -0.6235488474692223 | LOSS: 23.70683491230011
EPISODE: 5723 | TIME: 129.368346452713 s | REWARD: 7.067838372277388 | FRAMES: 928 | QMAX: 1373.9827 | EPSILON: 0.019750000000082396 | CTE: -0.6215898770109906 | LOSS: 19.822989851236343
EPISODE: 5724 | TIME: 129.64000725746155 s | REWARD: 5.322986232722962 | FRAMES: 926 | QMAX: 1250.0022 | EPSILON: 0.019750000000082396 | CTE: -1.4720641514369335 | LOSS: 29.742744266986847
EPISODE: 5725 | TIME: 129.30648231506348 s | REWARD: 14.

EPISODE: 5764 | TIME: 21.123574018478394 s | REWARD: 0.6440633291504207 | FRAMES: 80 | QMAX: 1242.087 | EPSILON: 0.019750000000082396 | CTE: -1.73348795653875 | LOSS: 37.28638231754303
EPISODE: 5765 | TIME: 20.494224071502686 s | REWARD: 2.793112212007275 | FRAMES: 73 | QMAX: 1274.1116 | EPSILON: 0.019750000000082396 | CTE: -1.4934781473767118 | LOSS: 40.361186265945435


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x00000214FAE8C9D0>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\weakref.py", line 346, in remove
    self = selfref()
KeyboardInterrupt: 
